In [6]:
from torch.utils.data import DataLoader
from torch import optim
from torch import nn

import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision
import torch
import cv2

import matplotlib.pyplot as plt
import numpy as np

from tqdm import tqdm

BATCH_SIZE = 40
EPOCH = 100
device = torch.device('mps')

In [2]:
transform = transforms.Compose([transforms.Resize(128), transforms.ToTensor()])

trainset = datasets.STL10(root='../data/', split='train', download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
testset = datasets.STL10(root='../data/', split='test', download=True, transform=transform)
testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
model = torchvision.models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs, 10)
model = model.to(device)
# model.load_state_dict(
#     torch.load('../models/stl10_resnet18.pth')
# )

/Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [4]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [7]:
for epoch in range(EPOCH + 1):
    running_loss, test_loss = 0.0, 0.0
    for data in tqdm(trainloader, desc=f'{epoch}th Training'):
        image, label = data[0].to(device), data[1].to(device)
        output = model(image)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        model.eval()
        with torch.no_grad():
            for data in testloader:
                test_image, test_label = data[0].to(device), data[1].to(device)
                output = model(test_image)
                loss = criterion(output, test_label)
                test_loss += loss.item()
        model.train()
    
    print(f"Epoch: {epoch}\tLoss : {running_loss / len(trainloader)}\tTest Loss : {test_loss / len(testloader)}")

0th Training: 100%|██████████| 125/125 [26:19<00:00, 12.63s/it]


Epoch: 0	Loss : 1.7674228773117064	Test Loss : inf


1th Training: 100%|██████████| 125/125 [22:57<00:00, 11.02s/it]


Epoch: 1	Loss : 1.4683975744247437	Test Loss : 224.9050468814373


2th Training:  46%|████▋     | 58/125 [10:39<12:17, 11.00s/it]

In [ ]:
def accuracyExprot(dataloader):
    correct, total = 0, 0
    model.eval()
    
    with torch.no_grad():
        for data in dataloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.detach(), 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print("Accuracy: %d%%" % (100 * correct / total))

accuracyExprot(trainloader)
accuracyExprot(testloader)

In [ ]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [ ]:
def cam(dataset, img_sample, img_size):
    model.eval()
    with torch.no_grad():
        model.layer4[1]